Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time


# Carregamento de dados

In [ ]:
# Não testei, não me apeteceu :)
dataset = pd.read_excel('Global_Superstore2.xlsx', index_col=0, comment='#') 

: 

# Tratamento de dados

# Modelação


Loading the training dataset